## WeRateDogs Data Wrangling project


## Table of Contents

- [Introduction](#intro)
- [Data Wrangling](#wrangling)
    - [The First Dataset: twitter-archive-enhanced](#first)
    - [The Second Dataset: Image Predictions File](#second)
    - [The Third  Dataset: Data via the Twitter API](#third)
- Exploratory Data Analysis
- [Conclusion](#Conclusion)



<a id='intro'></a>
## Introduction
The tweet archive of WeRateDogs on Twitter is analyzed here. WeRateDogs is a Twitter account that rates people's dogs with a humorous comment about the dog and it has over 4 million followers and has received international media coverage. Addithinal data such as retweet count and favorite count are obtained from Twitter's API. Predictions of breeds of dogs for each tweets are also provided.

<a id='wrangling'></a>
## Data Wrangling

In [157]:
#import the required libraries
import numpy as np
import pandas as pd
import requests
import os
import tweepy

<a id='first'></a>
# The First Dataset (twitter-archive-enhanced)

<a id='wrangling'></a>
## Gathering

In [158]:
twitter_archive = pd.read_csv('twitter-archive-enhanced.csv')

In [194]:
# View sapmle random 15 rows of twitter-archive DataFrame
twitter_archive.sample(2)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
205,852912242202992640,NaN,NaN,2017-04-14 15:51:39 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Meet Benny. He likes being adorable and making fun of you while you're on the trampoline. 12/10 let's help him out\r\n\r\nhttps://t.co/aVMjBqAy1x https://t.co/7gx2LksT3U,NaN,NaN,NaN,"https://www.gofundme.com/bennys-medical-bills,https://twitter.com/dog_rates/status/852912242202992640/photo/1,https://twitter.com/dog_rates/status/852912242202992640/photo/1",12,10,Benny,None,None,None,None
547,805520635690676224,NaN,NaN,2016-12-04 21:14:20 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Zeke the Wonder Dog. He never let that poor man keep his frisbees. One of the Spartans all time greatest receivers. 13/10 RIP Zeke https://t.co/zacX7S6GyJ,NaN,NaN,NaN,"https://twitter.com/dog_rates/status/805520635690676224/photo/1,https://twitter.com/dog_rates/status/805520635690676224/photo/1,https://twitter.com/dog_rates/status/805520635690676224/photo/1,https://twitter.com/dog_rates/status/805520635690676224/photo/1",13,10,Zeke,None,None,None,None


In [193]:
# expand column width to max
pd.set_option('display.max_colwidth', -1)

## Assessing

### twitter_archive columns:

- **tweet_id**: the unique identifier for each tweet                   
- **in_reply_to_status_id**         
- **in_reply_to_user_id**:          
- **timestamp**: time of tweet                   
- **source**: Utility used to post the Tweet                      
- **text**: tweet's text                         
- **retweeted_status_id**:  retweet ID         
- **retweeted_status_user_id**: retweet ID user ID     
- **retweeted_status_timestamp**:   time of retweet    
- **expanded_urls**:  tweet urls               
- **rating_numerator**:  actual rating of a dog. Almost always is greater than 10. 11/10, 12/10, 13/10, etc, because ["they're good dogs Brent"](https://knowyourmeme.com/memes/theyre-good-dogs-brent)           
- **rating_denominator**:  These ratings always have a denominator of 10.           
- **name**: The name of the dog                        
- **doggo**, **floofer**, **pupper** & **puppo**:  dog stages                      


In [160]:
twitter_archive.shape

(2356, 17)

In [161]:
twitter_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [162]:
# Check if there are any doplicated tweet_ids
len(twitter_archive.tweet_id.unique())

2356

In [163]:
twitter_archive.source.unique()

array(['<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
       '<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>',
       '<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>',
       '<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>'],
      dtype=object)

In [164]:
# Check if there are any doplicated Dogs' names
len(twitter_archive.name.unique())

957

In [165]:
twitter_archive.groupby("name").size().sort_values(ascending=False)
 

name
None         745
a             55
Charlie       12
Oliver        11
Lucy          11
Cooper        11
Lola          10
Tucker        10
Penny         10
Bo             9
Winston        9
Sadie          8
the            8
an             7
Toby           7
Daisy          7
Bailey         7
Buddy          7
Leo            6
Scout          6
Bella          6
Dave           6
Rusty          6
Jack           6
Jax            6
Milo           6
Koda           6
Stanley        6
Oscar          6
very           5
            ... 
Jiminus        1
Jimbo          1
Jim            1
Jett           1
Jessiga        1
Jessifer       1
Spencer        1
Jersey         1
Josep          1
Juckson        1
Kellogg        1
Julio          1
Sonny          1
Keet           1
Kayla          1
Kawhi          1
Katie          1
Kathmandu      1
Karma          1
Karll          1
Karl           1
Kara           1
Kanu           1
Kane           1
Kallie         1
Kaiya          1
Kaia           1
Sora     

In [166]:
twitter_archive.name.unique()

array(['Phineas', 'Tilly', 'Archie', 'Darla', 'Franklin', 'None', 'Jax',
       'Zoey', 'Cassie', 'Koda', 'Bruno', 'Ted', 'Stuart', 'Oliver',
       'Jim', 'Zeke', 'Ralphus', 'Canela', 'Gerald', 'Jeffrey', 'such',
       'Maya', 'Mingus', 'Derek', 'Roscoe', 'Waffles', 'Jimbo', 'Maisey',
       'Lilly', 'Earl', 'Lola', 'Kevin', 'Yogi', 'Noah', 'Bella',
       'Grizzwald', 'Rusty', 'Gus', 'Stanley', 'Alfy', 'Koko', 'Rey',
       'Gary', 'a', 'Elliot', 'Louis', 'Jesse', 'Romeo', 'Bailey',
       'Duddles', 'Jack', 'Emmy', 'Steven', 'Beau', 'Snoopy', 'Shadow',
       'Terrance', 'Aja', 'Penny', 'Dante', 'Nelly', 'Ginger', 'Benedict',
       'Venti', 'Goose', 'Nugget', 'Cash', 'Coco', 'Jed', 'Sebastian',
       'Walter', 'Sierra', 'Monkey', 'Harry', 'Kody', 'Lassie', 'Rover',
       'Napolean', 'Dawn', 'Boomer', 'Cody', 'Rumble', 'Clifford',
       'quite', 'Dewey', 'Scout', 'Gizmo', 'Cooper', 'Harold', 'Shikha',
       'Jamesy', 'Lili', 'Sammy', 'Meatball', 'Paisley', 'Albus',
       'Nept

In [167]:
twitter_archive.nunique()    

tweet_id                      2356
in_reply_to_status_id           77
in_reply_to_user_id             31
timestamp                     2356
source                           4
text                          2356
retweeted_status_id            181
retweeted_status_user_id        25
retweeted_status_timestamp     181
expanded_urls                 2218
rating_numerator                40
rating_denominator              18
name                           957
doggo                            2
floofer                          2
pupper                           2
puppo                            2
dtype: int64

In [168]:
# View descriptive statistics of twitter-archive DataFrame
twitter_archive.describe()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,rating_numerator,rating_denominator
count,2.356000e+03,7.800000e+01,7.800000e+01,1.810000e+02,1.810000e+02,2356.000000,2356.000000
mean,7.427716e+17,7.455079e+17,2.014171e+16,7.720400e+17,1.241698e+16,13.126486,10.455433
std,6.856705e+16,7.582492e+16,1.252797e+17,6.236928e+16,9.599254e+16,45.876648,6.745237
min,6.660209e+17,6.658147e+17,1.185634e+07,6.661041e+17,7.832140e+05,0.000000,0.000000
25%,6.783989e+17,6.757419e+17,3.086374e+08,7.186315e+17,4.196984e+09,10.000000,10.000000
50%,7.196279e+17,7.038708e+17,4.196984e+09,7.804657e+17,4.196984e+09,11.000000,10.000000
75%,7.993373e+17,8.257804e+17,4.196984e+09,8.203146e+17,4.196984e+09,12.000000,10.000000
max,8.924206e+17,8.862664e+17,8.405479e+17,8.874740e+17,7.874618e+17,1776.000000,170.000000


In [169]:
twitter_archive['rating_numerator'].isnull().sum() 

0

In [170]:
# Total number of records with zero rating_numerator
(twitter_archive['rating_numerator']== 0).sum()

2

In [171]:
#(twitter_archive['rating_numerator']== 0).sum()
twitter_archive['rating_numerator'].max()
# That will affect outlier

1776

In [172]:
(twitter_archive['rating_numerator']== 1776).sum()

1

In [192]:
# Show the information for the rating_numerator == 1776
twitter_archive.query("rating_numerator == 1776 ")

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
979,749981277374128128,NaN,NaN,2016-07-04 15:00:45 +0000,"<a href=""https://about.twitter.com/products/tweetdeck"" rel=""nofollow"">TweetDeck</a>",This is Atticus. He's quite simply America af. 1776/10 https://t.co/GRXwMxLBkh,NaN,NaN,NaN,https://twitter.com/dog_rates/status/749981277374128128/photo/1,1776,10,Atticus,None,None,None,None


In [174]:
twitter_archive.rating_numerator.unique()

array([  13,   12,   14,    5,   17,   11,   10,  420,  666,    6,   15,
        182,  960,    0,   75,    7,   84,    9,   24,    8,    1,   27,
          3,    4,  165, 1776,  204,   50,   99,   80,   45,   60,   44,
        143,  121,   20,   26,    2,  144,   88], dtype=int64)

In [175]:
twitter_archive.rating_denominator.unique()

array([ 10,   0,  15,  70,   7,  11, 150, 170,  20,  50,  90,  80,  40,
       130, 110,  16, 120,   2], dtype=int64)

In [176]:
twitter_archive.doggo.unique()

array(['None', 'doggo'], dtype=object)

In [177]:
twitter_archive.floofer.unique()

array(['None', 'floofer'], dtype=object)

In [178]:
twitter_archive.pupper.unique()

array(['None', 'pupper'], dtype=object)

In [179]:
twitter_archive.puppo.unique()

array(['None', 'puppo'], dtype=object)

### Quality

- Erroneous Datatype: tweet_id, timestamp
- source column contains <> tag
- There are retweets data
- Some expanded_urls contain more than one URL and some have missing value
- Invalid names
- rate checks
- Not needed columns:  in_reply_to_status_id  , in_reply_to_user_id, retweeted_status_id, retweeted_status_user_id, and retweeted_status_timestamp

### Tidiness
- **Dog Stages** (i.e doggo, floofer, pupper & puppo) should be one column

# The Second Dataset (Image Predictions File)

## Gathering

In [143]:
# Download Image Predictions File from Udacity's servers 
prediction = requests.get('https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv')

In [144]:
# check the request 
prediction.status_code

200

In [145]:
# save Predictions File
with open("image_predictions.tsv",mode="wb") as file:
    file.write(prediction.content)

In [146]:
# open the tsv file as a data frame
prediction=pd.read_csv("image_predictions.tsv",sep="\t")

In [147]:
prediction.sample(15)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
491,675534494439489536,https://pbs.twimg.com/media/CV_7CV6XIAEV05u.jpg,1,chow,0.749368,True,schipperke,0.133738,True,Newfoundland,0.049914,True
671,683111407806746624,https://pbs.twimg.com/media/CXrmMSpUwAAdeRj.jpg,1,cocker_spaniel,0.901392,True,soft-coated_wheaten_terrier,0.028605,True,miniature_schnauzer,0.017805,True
1584,797236660651966464,https://pbs.twimg.com/media/CxBafisWQAAtJ1X.jpg,2,collie,0.767005,True,Border_collie,0.100844,True,kelpie,0.048368,True
598,679722016581222400,https://pbs.twimg.com/media/CW7bkW6WQAAksgB.jpg,1,boxer,0.459604,True,Boston_bull,0.197913,True,French_bulldog,0.087023,True
2034,883838122936631299,https://pbs.twimg.com/media/DEQGFgAXUAAEvfi.jpg,1,Doberman,0.610946,True,miniature_pinscher,0.299603,True,kelpie,0.063020,True
1721,819588359383371776,https://pbs.twimg.com/media/C1_DQn3UoAIoJy7.jpg,1,Cardigan,0.547935,True,basenji,0.116442,True,Shetland_sheepdog,0.101681,True
1136,728760639972315136,https://pbs.twimg.com/media/Ch0T71OWMAA4yIw.jpg,1,Pembroke,0.939134,True,Cardigan,0.054336,True,Chihuahua,0.005590,True
1821,834574053763584002,https://pbs.twimg.com/media/C5UAqgyXAAAbMWH.jpg,1,toilet_tissue,0.262936,False,golden_retriever,0.226564,True,bathtub,0.078879,False
519,676496375194980353,https://pbs.twimg.com/media/CWNl3S9WcAARN34.jpg,1,pug,0.985387,True,Norwegian_elkhound,0.004417,True,French_bulldog,0.003893,True
1696,816450570814898180,https://pbs.twimg.com/media/C1SddosXUAQcVR1.jpg,1,web_site,0.352857,False,envelope,0.060107,False,nail,0.031291,False


## Assessing

## column names - prediction

- tweet_id: ID for each tweet
- jpg_url: image urls (1st image if several pictures are tweeted)
- img_num: number of image in a tweet
- p1,p2,p3: prediction of spieces/ num.1 prediction, num2 prediction, num3 prediction
- p1_conf,p2_conf,p3_conf: how confident the prediction is for each prediction
- p1_dog,p2_dog,p3_dog: if the prediction is dog or not


In [149]:
prediction.shape

(2075, 12)

In [150]:
prediction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [151]:
prediction.describe()

,tweet_id,img_num,p1_conf,p2_conf,p3_conf
count,2.075000e+03,2075.000000,2075.000000,2.075000e+03,2.075000e+03
mean,7.384514e+17,1.203855,0.594548,1.345886e-01,6.032417e-02
std,6.785203e+16,0.561875,0.271174,1.006657e-01,5.090593e-02
min,6.660209e+17,1.000000,0.044333,1.011300e-08,1.740170e-10
25%,6.764835e+17,1.000000,0.364412,5.388625e-02,1.622240e-02
50%,7.119988e+17,1.000000,0.588230,1.181810e-01,4.944380e-02
75%,7.932034e+17,1.000000,0.843855,1.955655e-01,9.180755e-02
max,8.924206e+17,4.000000,1.000000,4.880140e-01,2.734190e-01


# The Third  Dataset (Data via the Twitter API)

Back to the basic-ness of Twitter archives: retweet count and favorite count are two of the notable column omissions. Fortunately, this additional data can be gathered by anyone from Twitter's API. Well, "anyone" who has access to data for the 3000 most recent tweets, at least. But you, because you have the WeRateDogs Twitter archive and specifically the tweet IDs within it, can gather this data for all 5000+. And guess what? You're going to query Twitter's API to gather this valuable data.

## Gathering

## Assessing

## Cleaning

In [180]:
twitter_archive_clean = twitter_archive.copy()

### Quality

#### Define
- Change tweet_id Datatype from Intg to String
- Change timestamp Datatype from Integar to date format
- Solve the NaN values in expanded_urls column
- Drop Not needed columns: in_reply_to_status_id  , in_reply_to_user_id , retweeted_status_id, retweeted_status_user_id and retweeted_status_timestamp
- There are retweets data
Invalid names
rate checks




#### Code

In [181]:
# Change the data type of timestamp date from Integar to date format
twitter_archive_clean['timestamp'] = pd.to_datetime(twitter_archive_clean['timestamp']) 

In [182]:
# Change tweet_id Datatype from Intg to String
twitter_archive_clean['tweet_id'] = twitter_archive_clean.tweet_id.astype(str)

In [183]:
# set url for concatenating
url_front ="https://twitter.com/dog_rates/status/"

In [184]:
# if expanded urls col is na then add the defined url + tweet id for url
twitter_archive_clean.loc[twitter_archive_clean.expanded_urls.isna(),"expanded_urls"]=url_front + twitter_archive_clean["tweet_id"].map(str)

In [186]:
# Drop not needed columns: in_reply_to_status_id  , in_reply_to_user_id and retweeted
twitter_archive_clean.drop(['in_reply_to_status_id'  , 'in_reply_to_user_id', 'retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp'], axis=1, inplace=True)  

#### Test

In [189]:
twitter_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 12 columns):
tweet_id              2356 non-null object
timestamp             2356 non-null datetime64[ns, UTC]
source                2356 non-null object
text                  2356 non-null object
expanded_urls         2356 non-null object
rating_numerator      2356 non-null int64
rating_denominator    2356 non-null int64
name                  2356 non-null object
doggo                 2356 non-null object
floofer               2356 non-null object
pupper                2356 non-null object
puppo                 2356 non-null object
dtypes: datetime64[ns, UTC](1), int64(2), object(9)
memory usage: 221.0+ KB


In [190]:
twitter_archive_clean.sample(2)

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
267,841680585030541313,2017-03-14 16:01:03+00:00,"<a href=""http://twitter.com/download/iphone"" r...",This is Penny. She's a dragon slayer. Feared b...,https://twitter.com/dog_rates/status/841680585...,12,10,Penny,None,None,None,None
422,821813639212650496,2017-01-18 20:16:54+00:00,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: Meet Hercules. He can have what...,https://twitter.com/dog_rates/status/780601303...,12,10,Hercules,None,None,None,None


### Tidiness

#### Define
Compain Dog stages columns (i.e doggo, floofer, pupper & puppo) so that it has only one column for a dog stage

#### Code

In [132]:
# replace None with nan
twitter_archive_clean.replace("None",np.nan,inplace=True)

In [133]:
# replace nan with an empty space to concatnate strings
twitter_archive_clean[["doggo","floofer","pupper","puppo"]]=twitter_archive_clean[["doggo","floofer","pupper","puppo"]].fillna("")

In [134]:
# check the result
twitter_archive_clean.sample()

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
1578,687317306314240000,2016-01-13 16:56:30+00:00,"<a href=""http://twitter.com/download/iphone"" r...",This is Tyrone. He's a leaf wizard. Self-motiv...,https://twitter.com/dog_rates/status/687317306...,11,10,Tyrone,,,,


In [135]:
# concatnate dog stage columns to create a new column "stage"
twitter_archive_clean["stage"]=(twitter_archive_clean["doggo"] + twitter_archive_clean["floofer"] + twitter_archive_clean["pupper"] + twitter_archive_clean["puppo"])

In [136]:
# check unique stages
twitter_archive_clean.stage.unique()

array(['', 'doggo', 'puppo', 'pupper', 'floofer', 'doggopuppo',
       'doggofloofer', 'doggopupper'], dtype=object)

**There are tweets with multiple dog stages. It needs to be solved.**

In [137]:
# Handle multiple stages
twitter_archive_clean.loc[twitter_archive_clean.stage == 'doggopupper', 'stage'] = 'doggo,pupper'
twitter_archive_clean.loc[twitter_archive_clean.stage == 'doggopuppo', 'stage'] = 'doggo,puppo'
twitter_archive_clean.loc[twitter_archive_clean.stage == 'doggofloofer', 'stage'] = 'doggo,floofer'

# Handle missing values through change empty stages to na
twitter_archive_clean.loc[twitter_archive_clean.stage == '', 'stage'] = np.nan

In [138]:
# check the result
twitter_archive_clean[twitter_archive_clean["stage"].notna()].stage.unique()

array(['doggo', 'puppo', 'pupper', 'floofer', 'doggo,puppo',
       'doggo,floofer', 'doggo,pupper'], dtype=object)

In [139]:
# drop "doggo","floofer","pupper","puppo" columns
twitter_archive_clean.drop(columns=["doggo","floofer","pupper","puppo"],axis=1,inplace=True)

In [140]:
# reset index
twitter_archive_clean.reset_index(inplace=True,drop=True)

## Test

In [141]:
# check number of observations
twitter_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 9 columns):
tweet_id              2356 non-null int64
timestamp             2356 non-null datetime64[ns, UTC]
source                2356 non-null object
text                  2356 non-null object
expanded_urls         2356 non-null object
rating_numerator      2356 non-null int64
rating_denominator    2356 non-null int64
name                  1611 non-null object
stage                 380 non-null object
dtypes: datetime64[ns, UTC](1), int64(3), object(5)
memory usage: 165.7+ KB


In [142]:
# check data structure
twitter_archive_clean.sample(2)

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,stage
326,833826103416520705,2017-02-20 23:50:09+00:00,"<a href=""http://twitter.com/download/iphone"" r...",Meet Charlie. She asked u to change the channe...,https://twitter.com/dog_rates/status/833826103...,13,10,Charlie,NaN
135,866450705531457537,2017-05-22 00:28:40+00:00,"<a href=""http://twitter.com/download/iphone"" r...",This is Jamesy. He gives a kiss to every other...,https://twitter.com/dog_rates/status/866450705...,13,10,Jamesy,pupper


In [85]:
# make pick up checks with the original data
twitter_archive_clean[twitter_archive_clean.tweet_id ==881536004380872706].stage

56    pupper
Name: stage, dtype: object

In [86]:
twitter_archive[twitter_archive.tweet_id ==881536004380872706].pupper

56    pupper
Name: pupper, dtype: object

## Cleaning

In [95]:
prediction_clean = prediction.copy()

### Define

### Code

### Test

## Cleaning